In [1]:
import h5py
import nibabel as nib
import numpy as np
import os
import math
import PIL
from PIL import Image
import sys

In [2]:
volumeDirs = ["Dataset/TrainVolumes/", "Dataset/ValidationVolumes/", "Dataset/TestVolumes/"]
segmentationDirs = ["Dataset/TrainSegmentations/", "Dataset/ValidationSegmentations/", "Dataset/TestSegmentations/"]

fileNames = ["TrainDataset.hdf5", "ValidationDataset.hdf5", "TestDataset.hdf5"]

numFiles = []

for fileName in volumeDirs:
    numFiles.append(len([name for name in os.listdir(fileName) if os.path.isfile(os.path.join(fileName, name))]))

#Percent of slices to keep from each scan, starts from middle of array
keepRate = 0.3

#Resize all slices/segmentations to imageDim x imageDim
imageDim = 256

In [3]:
def window_level(vol, window_center, window_width): 
    img_min = window_center - window_width // 2 
    img_max = window_center + window_width // 2 
    window_image = vol.copy() 
    window_image[window_image < img_min] = img_min 
    window_image[window_image > img_max] = img_max 

    return window_image 

In [4]:
livers = []
total = []

minVal = float('inf')
maxVal = float('-inf')

for volumeDir in volumeDirs:
    for i, name in enumerate(os.listdir(volumeDir)):
        #Disregards hidden files
        if name[0] == '.':
            continue

        ctScan = nib.load(volumeDir + name)
        volumeData = ctScan.get_fdata()

        minVal = min(np.amin(volumeData), minVal)
        maxVal = max(np.amax(volumeData), maxVal)

for i in range(len(volumeDirs)):
    volumeDir = volumeDirs[i]
    segmentationDir = segmentationDirs[i]

    numLivers = 0
    totalSlices = 0

    file = h5py.File(fileNames[i], "w")

    sliceNum = 0

    for i, name in enumerate(os.listdir(volumeDir)):
        #Disregards hidden files
        if name[0] == '.':
            continue

        #Loads segmentation and volume data from .nii file
        ctScan = nib.load(volumeDir + name)
        volumeData = ctScan.get_fdata()

        volumeData = window_level(volumeData, 40, 400)

        segmentation = nib.load(segmentationDir + os.listdir(segmentationDir)[i])
        segmentData = segmentation.get_fdata()

        #Loops through all usable slices and adds data to h5 file
        #Finds middle index, subtracts half * keepRate from it, goes to middle index + half * keepRate
        for plane in range(math.ceil(((volumeData.shape[2] - 1) / 2) - (((volumeData.shape[2] - 1) / 2) * keepRate)), 
        math.floor(((volumeData.shape[2] - 1) / 2) + (((volumeData.shape[2] - 1) / 2) * keepRate))):

            volumeSlice = np.array(Image.fromarray(volumeData[:,:,plane].astype(np.int16)).resize((imageDim, imageDim), Image.BILINEAR))
            segmentSlice = segmentData[:,:,plane].astype(np.int16)

            volumeSlice = volumeSlice.astype(np.float16)

            volumeSlice -= minVal
            volumeSlice /= maxVal - minVal

            #Gets max value of current segmenation, limits it to 1 (1 if contains liver, 0 if not)
            label = min(np.amax(segmentSlice), 1)
            segmentSlice = np.array(Image.fromarray(segmentSlice).resize((imageDim, imageDim), Image.NEAREST))

            numLivers += label
            totalSlices += 1

            #Creates subgroup for current slice in current scan, adds slice/segmentation/label data
            currSlice = file.create_group("Slice" + str(sliceNum))
            currSlice.create_dataset("Slice", data=volumeSlice)
            currSlice.create_dataset("Segmentation", data=segmentSlice)
            currSlice.attrs.create("ImageLabel", label, (1,), "int")

            sliceNum += 1

    livers.append(numLivers)
    total.append(totalSlices)

print(f"Liver Present: {livers} Total: {total}")

file.close()

KeyboardInterrupt: 

In [6]:
for fileName in fileNames:
    dataFile = h5py.File(fileName, 'r')
    print(list(dataFile.keys()))
        
    for slice in dataFile:
        print(dataFile[slice]["Slice"])
        print(dataFile[slice]["Segmentation"])
        print(dataFile[slice].attrs.get("ImageLabel"))

dataFile.close()

['Slice0', 'Slice1', 'Slice10', 'Slice100', 'Slice1000', 'Slice1001', 'Slice1002', 'Slice1003', 'Slice1004', 'Slice1005', 'Slice1006', 'Slice1007', 'Slice1008', 'Slice1009', 'Slice101', 'Slice1010', 'Slice1011', 'Slice1012', 'Slice1013', 'Slice1014', 'Slice1015', 'Slice1016', 'Slice1017', 'Slice1018', 'Slice1019', 'Slice102', 'Slice1020', 'Slice1021', 'Slice1022', 'Slice1023', 'Slice1024', 'Slice1025', 'Slice1026', 'Slice1027', 'Slice1028', 'Slice1029', 'Slice103', 'Slice1030', 'Slice1031', 'Slice1032', 'Slice1033', 'Slice1034', 'Slice1035', 'Slice1036', 'Slice1037', 'Slice1038', 'Slice1039', 'Slice104', 'Slice1040', 'Slice1041', 'Slice1042', 'Slice1043', 'Slice1044', 'Slice1045', 'Slice1046', 'Slice1047', 'Slice1048', 'Slice1049', 'Slice105', 'Slice1050', 'Slice1051', 'Slice1052', 'Slice1053', 'Slice1054', 'Slice1055', 'Slice1056', 'Slice1057', 'Slice1058', 'Slice1059', 'Slice106', 'Slice1060', 'Slice1061', 'Slice1062', 'Slice1063', 'Slice1064', 'Slice1065', 'Slice1066', 'Slice1067', 

KeyboardInterrupt: 